In [1]:
%%time

import tensorflow as tf
print(tf.__version__)
import sys
sys.path.append("..")
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


def plot_loss(history, *losses):
    for loss in losses:
        plt.plot(history.history[loss], label=loss)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

def scaling(x, min, max):
    return np.where(x < min, 0.0, np.where(x > max, 1.0, (x - min) / (max - min)))

early_stopping = EarlyStopping(
    monitor='val_loss',  # 
    patience=50,        # 
    verbose=1,          # 
    mode='min',         # 
    restore_best_weights=True  # 
)
from keras.callbacks import Callback


2024-11-24 16:32:05.388206: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 16:32:05.433614: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 16:32:05.433638: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 16:32:05.433669: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 16:32:05.441753: I tensorflow/core/platform/cpu_feature_g

2.14.0
CPU times: user 2.85 s, sys: 2.28 s, total: 5.14 s
Wall time: 2.77 s


In [2]:
SAVE_DIR = "../data"
file_criteo = SAVE_DIR + "/criteo-uplift-v2.1.csv"
df_criteo_ori = pd.read_csv(file_criteo, sep=',')

In [11]:


# 0
model_name = "DCL-new"
result_list = []
DRP_aucc_test_list = []
roi_rank_pre_test_list = []

#sample_list = [0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21]

lambda_para = [0.2]

sample = 1.0
count = 20
for i in range(count):
    random_state=20220720
    df_criteo=df_criteo_ori.sample(frac=sample, random_state=random_state).reset_index(drop=True)
    X = df_criteo[['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']].values

    X[:, 0] = scaling(X[:, 0], min=np.min(X[:, 0]), max=np.max(X[:, 0]))
    X[:, 1] = scaling(X[:, 1], min=np.min(X[:, 1]), max=np.max(X[:, 1]))
    X[:, 2] = scaling(X[:, 2], min=np.min(X[:, 2]), max=np.max(X[:, 2]))
    X[:, 3] = scaling(X[:, 3], min=np.min(X[:, 3]), max=np.max(X[:, 3]))
    X[:, 4] = scaling(X[:, 4], min=np.min(X[:, 4]), max=np.max(X[:, 4]))
    X[:, 5] = scaling(X[:, 5], min=np.min(X[:, 5]), max=np.max(X[:, 5]))
    X[:, 6] = scaling(X[:, 6], min=np.min(X[:, 6]), max=np.max(X[:, 6]))
    X[:, 7] = scaling(X[:, 7], min=np.min(X[:, 7]), max=np.max(X[:, 7]))
    X[:, 8] = scaling(X[:, 8], min=np.min(X[:, 8]), max=np.max(X[:, 8]))
    X[:, 9] = scaling(X[:, 9], min=np.min(X[:, 9]), max=np.max(X[:, 9]))
    X[:, 10] = scaling(X[:, 10], min=np.min(X[:, 10]), max=np.max(X[:, 10]))
    X[:, 11] = scaling(X[:, 11], min=np.min(X[:, 11]), max=np.max(X[:, 11]))

    T = df_criteo['treatment'].values.reshape(-1, 1)
    Y_visit = df_criteo['visit'].values.reshape(-1, 1)
    Y_conv = df_criteo['conversion'].values.reshape(-1, 1)
    T.shape, Y_visit.shape, Y_conv.shape

    # calculate len
    train_len = int(len(X) * 0.70)
    cali_len = int(len(X) * 0.0)
    test_len = len(X) - train_len - cali_len

    # obtain train set
    X_train = X[:train_len, :]
    T_train = T[:train_len, :]
    Y_visit_train = Y_visit[:train_len, :]
    Y_conv_train = Y_conv[:train_len, :]

    # obtain calibration set
    X_cali = X[train_len:train_len+cali_len, :]
    T_cali = T[train_len:train_len+cali_len, :]
    Y_visit_cali = Y_visit[train_len:train_len+cali_len, :]
    Y_conv_cali = Y_conv[train_len:train_len+cali_len, :]

    # obtain test set
    X_test = X[train_len+cali_len:, :]
    T_test = T[train_len+cali_len:, :]
    Y_visit_test = Y_visit[train_len+cali_len:, :]
    Y_conv_test = Y_conv[train_len+cali_len:, :]

    #print(train_len, X_train.shape, X_test.shape, len(X), X_cali.shape, T_train.shape, Y_visit_train.shape)
    
    # train
    
    sys.path.append("..")
    from model.uplift_model import *

    count = 1
    # 2.5e-5
    import keras
    import keras.backend as K
    import tensorflow as tf
    from keras.callbacks import LearningRateScheduler
    from keras.callbacks import ReduceLROnPlateau
    from model.roi_model import *
    
    final_model = get_roi_rank_criteo_model_reward(lambda_para[i])
    final_model.compile(loss=None, optimizer='adam')

    
    mcp_save = ModelCheckpoint('../model_file/roi/criteo/final_model/roi_rank/{}_{}.h5'.format(model_name, sample), save_best_only=False, monitor='val_loss', mode='min', save_weights_only=True)
    history = final_model.fit([X_train, T_train, Y_conv_train, Y_visit_train], validation_split=0.2, epochs=6000, batch_size=train_len, shuffle=True, verbose=0, callbacks=[mcp_save])
    #plot_loss(history, "loss", "val_loss", "obj", "val_obj")
    
    # predict
    import sklearn 
    import sklearn.metrics
    from metric.Metric import *
    import keras
    import keras.backend as K
    import tensorflow as tf
    from keras.callbacks import LearningRateScheduler
    from keras.callbacks import ReduceLROnPlateau
    from model.roi_model import *
    
    final_model = get_roi_rank_criteo_model_reward(lambda_para[i])
    final_model.load_weights('../model_file/roi/criteo/final_model/roi_rank/{}_{}.h5'.format(model_name, sample))

    
    
    roi_rank_pre_test = final_model.predict([X_test, T_test, Y_conv_test, Y_visit_test])
    roi_rank_pre_test_list.append(roi_rank_pre_test)
    DRP_aucc = get_uplift_model_aucc_no_show(t=(T_test > 0.5).flatten(), y_reward=Y_conv_test.flatten(), y_cost=Y_visit_test.flatten(), roi_pred=roi_rank_pre_test.flatten(), quantile=200)
    DRP_aucc_test_list.append(DRP_aucc)
    result_list.append(DRP_aucc[0])
    
    i = i + 1    



    
mean_pred = np.mean(result_list, axis=0)
std_pred = np.std(result_list, axis=0)


print(f"mean = {mean_pred:.4f}")
print(f"std = {std_pred:.4f}")


# store test aucc for pic 
import pandas as pd

def get_aucc_cost_curve(aucc_list):
    delta_cost_list_group = np.array([aucc[1] for aucc in aucc_list])
    delta_reward_list_group = np.array([aucc[2] for aucc in aucc_list])
    
    avg_delta_cost_list = np.mean(delta_cost_list_group, axis=0)
    avg_delta_reward_list = np.mean(delta_reward_list_group, axis=0)
    
    df_aucc_cost_curve = pd.DataFrame(avg_delta_cost_list, columns=['delta_cost'])
    df_aucc_cost_curve['delta_reward'] = avg_delta_reward_list
    
    return df_aucc_cost_curve




mean = 0.7821
std = 0.0010
